Ok avec `PoinNet0_env`

Applique le script get_cirtical_points.py à tous les fichiers .off d'un répertoire (`folder_path+dataset+"/"`) et enregistre les outputs dans le dossier `output_path`

In [6]:
import os
import glob
from tqdm import tqdm
from pathlib import Path

Dataset_path = '/home/pelissier/These-ATER/Papier_international3/Dataset/'
Critical_pts_folder_path = '/home/pelissier/These-ATER/Papier_international3/PointNet/my_PointNet/Critical_pts/'

In [7]:
def get_off_files_recursive(folder_path, extension='*.off'):
    # Get all .off files from the folder and its subfolders
    off_files = glob.glob(os.path.join(folder_path, "**", extension), recursive=True)
    
    # Filter out files that contain 'SIMPL' in their name
    off_files = [file for file in off_files if 'SMPL' not in os.path.basename(file)]
    
    return off_files

def count_files_in_folder_recursive(folder_path):
    file_count = 0
    for _, _, files in os.walk(folder_path):
        file_count += len(files)
    return file_count

## ModelNet40

In [8]:
# Usage example
dataset = "ModelNet40"
# Tous les path des fichiers .off de ModelNet40
off_files = get_off_files_recursive(Dataset_path+dataset+"/"); print(f"Il y a : {len(off_files)} fichiers .off dans le dataset {dataset}")

# Creation du dossier de sortie : Critical_pts/ModelNet40
# Check if folder exists, and if not, create it
modelnet40_path = Critical_pts_folder_path+dataset+"/"
if not Path(modelnet40_path).exists(): Path(modelnet40_path).mkdir(parents=True, exist_ok=True)
else : print("Le dossier "+dataset+" existe déjà !")

Il y a : 12311 fichiers .off dans le dataset ModelNet40
Le dossier ModelNet40 existe déjà !


In [9]:
# Les catégories de ModelNet40 
categories = os.listdir(Dataset_path+dataset+"/"); print(len(categories))

# Create one folder for each category in the output folder : Critical_pts/ModelNet40/category
for category in categories:
    category_folder = os.path.join(modelnet40_path, category)
    if not Path(category_folder).exists():Path(category_folder).mkdir(parents=True, exist_ok=True)
    #else: print(f"Le dossier {category} existe déjà !")

40


In [10]:
import os
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed

# Path to the toto.py script
kind_of_outputs = '{"critical and non-critical points": true, "only critical points": true, "objet": true}'
affichage = "True"  # or "False", as required

# Path to save failed file names
failed_files_log = Critical_pts_folder_path+'failed_files.txt'


def run_script(off_file):
    category = "_".join(os.path.basename(off_file).split('_')[:-1])
    category_folder = os.path.join(Critical_pts_folder_path, dataset, category)+'/'
    
    cmd = [
        "python", 
        "/home/pelissier/These-ATER/Papier_international3/PointNet/my_PointNet/get_critical_points.py", 
        off_file, 
        category_folder, 
        "--kind_of_outputs", kind_of_outputs, 
        "--affichage", affichage
    ]
    #try:
    # Capture output and error, prevent displaying errors
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    # Check if the process succeeded (returncode 0 means success)
    if result.returncode != 0:
        # If there is an error, return the file name to log it
        return off_file

    # except Exception as e:
    #     # Log the file if any exception occurs
    #     return off_file
    
    return None  # No errors

In [11]:
import concurrent.futures

failed_files = []

# Using ThreadPoolExecutor for parallel execution
with concurrent.futures.ProcessPoolExecutor() as executor:  # Adjust max_workers as needed
    # Submit tasks to the executor
    futures = {executor.submit(run_script, off_file): off_file for off_file in off_files}
    
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        if i%250==0:
            print(f"Objectif : {(len(off_files)-len(failed_files))*5}, Créé : {count_files_in_folder_recursive(modelnet40_path)}, Pbl : {len(failed_files)}")
        try:
            result = future.result()
            if result is not None:
                failed_files.append(result)
        except Exception as exc:
            pass#print(exc)

# Save the failed file names to a log file
if failed_files:
    with open(failed_files_log, 'w') as f:
        for file in failed_files:
            f.write(f"{file}\n")
else : 
    with open(failed_files_log, 'w') as f : f.write("Tout est ok")


Objectif : 61555, Créé : 13, Pbl : 0
Objectif : 61555, Créé : 1265, Pbl : 0
Objectif : 61555, Créé : 2507, Pbl : 0
Objectif : 61555, Créé : 3760, Pbl : 0
Objectif : 61555, Créé : 5013, Pbl : 0
Objectif : 61555, Créé : 6269, Pbl : 0
Objectif : 61555, Créé : 7510, Pbl : 0
Objectif : 61555, Créé : 8763, Pbl : 0
Objectif : 61555, Créé : 10011, Pbl : 0
Objectif : 60830, Créé : 10525, Pbl : 145
Objectif : 60125, Créé : 11075, Pbl : 286
Objectif : 60125, Créé : 12330, Pbl : 286
Objectif : 58960, Créé : 12405, Pbl : 519
Objectif : 57710, Créé : 12405, Pbl : 769
Objectif : 56460, Créé : 12405, Pbl : 1019
Objectif : 56225, Créé : 13435, Pbl : 1066
Objectif : 55110, Créé : 13555, Pbl : 1289
Objectif : 53860, Créé : 13555, Pbl : 1539
Objectif : 53400, Créé : 14360, Pbl : 1631
Objectif : 53400, Créé : 15618, Pbl : 1631
Objectif : 53400, Créé : 16855, Pbl : 1631
Objectif : 52350, Créé : 17045, Pbl : 1841
Objectif : 51100, Créé : 17045, Pbl : 2091
Objectif : 49850, Créé : 17045, Pbl : 2341
Objectif :

# Autre dataset

In [ ]:
a

In [2]:
import concurrent.futures
import subprocess

# Define the output path and other arguments
output_path = "/home/pelissier/These-ATER/Papier_international3/PointNet/my_PointNet/Critical_pts/ModelNet40/"
kind_of_outputs = '{"critical and non-critical points": true, "only critical points": true, "objet": true}'
affichage = "True"  # or "False", as required

# Function to run the script
def run_script(input_path):
    command = [
        "python", 
        "/home/pelissier/These-ATER/Papier_international3/PointNet/my_PointNet/get_critical_points.py", 
        input_path, 
        output_path, 
        "--kind_of_outputs", kind_of_outputs, 
        "--affichage", affichage
    ]

    # Use subprocess to execute the command
    subprocess.run(command, capture_output=True, text=True)

        

# # Use ProcessPoolExecutor with tqdm for progress tracking
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     # Create a tqdm progress bar
#     with tqdm(total=len(off_files), desc="Processing .off files") as pbar:
#         # Submit tasks to the executor
#         futures = {executor.submit(run_script, file): file for file in off_files}
        
#         # Update the progress bar as each task is completed
#         for future in concurrent.futures.as_completed(futures):
#             pbar.update(1)